## Análisis de tipos de producto CvLAC

In [1]:
import pandas as pd
import requests
import time
import json
dtype={
'COD_RH':str,
'NRO_ID_GRUPO':str,
'COD_RH_MUNICIPIO':str,
'COD_INST':str,
'COD_INST_AVALA':str,
'COD_INST_MACRO':str,
'COD_INSTITUCION':str,         
'COD_INST_OTRO':str,
'ID_DEPARTAMENTO':str,
'COD_DEPA_DANE': str,
'COD_MUNI_DANE':str,
'COD_RH_OTRO':str,
'COD_RH_COAUTOR':str,
'COD_RH_CREA':str
 }

In [2]:
#cp product_type.json product_type_bak.json

In [3]:
f=open('product_type.json','r')
l=json.load(f)
f.close()

In [4]:
#cp product_type.json product_type_bak.json

In [5]:
#df=df[:176400]
#f=open('product_type_bak.json','w')
#json.dump(df,f)
#f.close()

In [4]:
len(l)

340915

In [5]:
l[0]

{'COD_RH': '0000000011',
 'COD_PRODUCTO': 1,
 'SGL_CATEGORIA': None,
 'product_type': [{'COD_TIPO_PRODUCTO': '63',
   'TXT_NME_TIPO_PRODUCTO': 'Monografía de conclusión de curso de perfeccionamiento/especialización',
   'NRO_NIVEL': 1,
   'COD_TIPO_PRODUCTO_PADRE': '6',
   'STA_ACTIVO': 'T',
   'product_type': [{'COD_TIPO_PRODUCTO': '6',
     'TXT_NME_TIPO_PRODUCTO': 'Trabajos dirigidos/Tutorías',
     'NRO_NIVEL': 0,
     'STA_ACTIVO': 'T'}]}]}

### Extraer niveles

y convertir a Pandas

In [7]:
for i in range(len(l)):
    d=l[i].copy()
    dd=l[i]
    while d.get('product_type'):
        if i%100==0:
            print(i,end='\r')
        #print(d.get('product_type'))
        d=d.get('product_type')[0]
        dd[f'NIVEL_{d.get("NRO_NIVEL")}']=f'{d.get("COD_TIPO_PRODUCTO")}: {d.get("TXT_NME_TIPO_PRODUCTO")}'
    NIVEL_MAX=sorted([x for x in dd.keys() if x.find('NIVEL_')>-1])[-1]
    if isinstance(NIVEL_MAX,str):
        dd['NIVEL_MAX']=dd[NIVEL_MAX]        

In [8]:
pf=pd.DataFrame(l)

### Contar niveles

In [9]:
len(pf['NIVEL_0'].unique())

9

In [10]:
len(pf['NIVEL_1'].unique())

68

In [11]:
len(pf['NIVEL_2'].unique())

165

In [12]:
len(pf['NIVEL_3'].unique())

10

In [13]:
len(pf['NIVEL_MAX'].unique())

206

In [14]:
len(pf['SGL_CATEGORIA'].unique())

117

### Extraer tipos de productos únicos

y ordenar por `'SGL_CATEGORIA'` y `'NIVEL_MAX'` para seleccionar ejemplos de valores no vacíos de `'SGL_CATEGORIA'`

In [15]:
pfu=pf.drop('product_type',axis='columns'
           ).drop_duplicates(
   subset=[x for x in pf.columns if x not in ['COD_RH','COD_PRODUCTO','product_type' ] ]  
    ).sort_values(['SGL_CATEGORIA','NIVEL_MAX']).drop_duplicates().reset_index(drop=True)#.fillna('')

In [16]:
pfu.shape

(906, 8)

In [17]:
pfu['CATEGORIAS']=pfu['NIVEL_MAX'].apply(lambda s: 
   pfu[pfu.fillna(False)['NIVEL_MAX']==s]['SGL_CATEGORIA'].dropna().str.split('_').str[0
                    ].unique()                      
                     ).apply(lambda l:[ x for x in l if x])

In [ ]:
pfu.sha

### Remover duplicados y visualizar resultado

In [18]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth',200)

In [19]:
pfu['NÚMERO']=pfu['NIVEL_MAX'].apply(lambda s:pf[pf['NIVEL_MAX']==s].shape[0])

In [20]:
pfu=pfu.drop_duplicates(subset='NIVEL_MAX').fillna('').sort_values(['NIVEL_0','NIVEL_1','NIVEL_2']).reset_index(drop=True)
pfu[[x for x in pfu.columns if x not in ['COD_RH','COD_PRODUCTO'] ]]

,SGL_CATEGORIA,NIVEL_1,NIVEL_0,NIVEL_MAX,NIVEL_2,NIVEL_3,CATEGORIAS,NÚMERO
0,ART-00,11: Artículo,1: Producción bibliográfica,111: Publicado en revista especializada,111: Publicado en revista especializada,,"[ART-00, ART-ART, ART-GC, EPA-EPA, EPA-EX, PE-00, PE-PE, PF-00, PF-EX, PF-PF, PIC-PIC, PID-00, PID-EX, PID-PID, PPC-PPC]",65340
1,ART-00,11: Artículo,1: Producción bibliográfica,112: Corto (Resumen),112: Corto (Resumen),,"[ART-00, ART-ART, ART-GC, PE-00, PE-PE, PF-00, PF-EX, PF-PF, PID-00, PID-PID]",5399
2,ART-00,11: Artículo,1: Producción bibliográfica,113: Revisión (Survey),113: Revisión (Survey),,"[ART-00, ART-ART, ART-GC, PE-PE, PF-00, PF-EX, PID-00, PID-PID]",2849
3,ART-00,11: Artículo,1: Producción bibliográfica,114: Caso clínico,114: Caso clínico,,"[ART-00, ART-ART, ART-GC, PE-PE, PF-00, PID-00, PID-PID]",1058
4,,12: Trabajos en eventos (Capítulos de memoria),1: Producción bibliográfica,121: Completo,121: Completo,,[],21833
5,,12: Trabajos en eventos (Capítulos de memoria),1: Producción bibliográfica,122: Resumen,122: Resumen,,[],15464
6,PE-PE,13: Libro,1: Producción bibliográfica,131: Otro libro publicado,131: Otro libro publicado,,"[PE-PE, PF-00, PF-EX, PID-00, PID-PID]",2040
7,PE-PE,13: Libro,1: Producción bibliográfica,132: Capítulo de libro,132: Capítulo de libro,,"[PE-PE, PF-00, PF-EX, PF-PF, PID-00, PID-PID]",5947
8,PE-PE,13: Libro,1: Producción bibliográfica,133: Libro pedagógico y/o de divulgación,133: Libro pedagógico y/o de divulgación,,"[PE-PE, PF-00, PID-00, PID-PID]",829
9,PE-PE,13: Libro,1: Producción bibliográfica,134: Libro resultado de investigación,134: Libro resultado de investigación,,"[PE-PE, PF-00, PF-EX, PIC-PIC, PID-00, PID-EX, PID-PID]",4983


In [21]:
pfu['NÚMERO'].sum()

340915

In [22]:
pfu.columns

Index(['COD_RH', 'COD_PRODUCTO', 'SGL_CATEGORIA', 'NIVEL_1', 'NIVEL_0',
       'NIVEL_MAX', 'NIVEL_2', 'NIVEL_3', 'CATEGORIAS', 'NÚMERO'],
      dtype='object')

In [23]:
pfu[['SGL_CATEGORIA', 'NIVEL_0','NIVEL_1','NIVEL_2', 'NIVEL_3',
       'NIVEL_MAX', 'NÚMERO','CATEGORIAS']].to_excel('productos_scienti.xlsx',index=False)

In [24]:
pf[['COD_RH', 'COD_PRODUCTO', 'SGL_CATEGORIA', 'product_type','NIVEL_MAX']].fillna('').to_json('product_type_NIVEL_MAX.json',orient='records')